In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
#Open dataframes
dataframeOrdem = pd.read_excel("product_data.xlsx")
dataframeProduto = pd.read_excel("product_data.xlsx")

#Order do not have product category
dataframeOrdem.head(5)

,ProductKey,SKU,Product,Standard Cost,Color,List Price,Model,Subcategory,Category
0,210.0,FR-R92B-58,"HL Road Frame - Black, 58",868.6342,Black,1431.5000,HL Road Frame,Road Frames,Components
1,211.0,FR-R92R-58,"HL Road Frame - Red, 58",868.6342,Red,1431.5000,HL Road Frame,Road Frames,Components
2,212.0,HL-U509-R,"Sport-100 Helmet, Red",12.0278,Red,33.6442,Sport-100,Helmets,Accessories
3,213.0,HL-U509-R,"Sport-100 Helmet, Red",13.8782,Red,33.6442,Sport-100,Helmets,Accessories
4,214.0,HL-U509-R,"Sport-100 Helmet, Red",13.0863,Red,34.9900,Sport-100,Helmets,Accessories


In [6]:
#Creating a new dataframe
dataframeMerged = pd.merge(dataframeOrdem, dataframeProduto, left_on='Product', right_on='Product').dropna()

dataframeMerged.head(5)

,ProductKey_x,SKU_x,Product,Standard Cost_x,Color_x,List Price_x,Model_x,Subcategory_x,Category_x,ProductKey_y,SKU_y,Standard Cost_y,Color_y,List Price_y,Model_y,Subcategory_y,Category_y
0,210.0,FR-R92B-58,"HL Road Frame - Black, 58",868.6342,Black,1431.5000,HL Road Frame,Road Frames,Components,210.0,FR-R92B-58,868.6342,Black,1431.5000,HL Road Frame,Road Frames,Components
1,211.0,FR-R92R-58,"HL Road Frame - Red, 58",868.6342,Red,1431.5000,HL Road Frame,Road Frames,Components,211.0,FR-R92R-58,868.6342,Red,1431.5000,HL Road Frame,Road Frames,Components
2,212.0,HL-U509-R,"Sport-100 Helmet, Red",12.0278,Red,33.6442,Sport-100,Helmets,Accessories,212.0,HL-U509-R,12.0278,Red,33.6442,Sport-100,Helmets,Accessories
3,212.0,HL-U509-R,"Sport-100 Helmet, Red",12.0278,Red,33.6442,Sport-100,Helmets,Accessories,213.0,HL-U509-R,13.8782,Red,33.6442,Sport-100,Helmets,Accessories
4,212.0,HL-U509-R,"Sport-100 Helmet, Red",12.0278,Red,33.6442,Sport-100,Helmets,Accessories,214.0,HL-U509-R,13.0863,Red,34.9900,Sport-100,Helmets,Accessories


In [15]:
#Grouping dataframe by product and transforming it from pandas object to list, keeping only category names
grouped = dataframeMerged.groupby("Product")
productOfGrouped = grouped['subcategory_x'].apply(list).to_list()
productOfGrouped[0:5]

[['Caps', 'Caps', 'Caps', 'Caps', 'Caps', 'Caps', 'Caps', 'Caps', 'Caps'],
 ['Chains'],
 ['Vests'],
 ['Vests'],
 ['Vests']]

In [17]:
#Deleting orders one item only in order to increase both confidence and lift
for lista in productOfGrouped:
    if len(lista) == 1:
        productOfGrouped.remove(lista)

In [18]:
#One hot encoding

from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
teArray = te.fit(productOfGrouped).transform(productOfGrouped)
teDataframe = pd.DataFrame(teArray, columns=te.columns_)

teDataframe.head()

,Bib-Shorts,Caps,Derailleurs,Gloves,Helmets,Jerseys,Mountain Bikes,Mountain Frames,Pedals,Road Bikes,Road Frames,Shorts,Socks,Tights,Touring Bikes,Touring Frames,Vests
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [19]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(teDataframe, min_support = 0.00005, use_colnames = True) 
frequent_itemsets.sort_values(by=['support'], ascending = True).head(10)

#Support is defined as frequency that both items occur together in the basket divided by total of transactions in the dataset
#Since there are too many orders with no category variability, support is low

,support,itemsets
0,0.009091,(Bib-Shorts)
13,0.009091,(Tights)
12,0.009091,(Socks)
16,0.009091,(Vests)
2,0.009091,(Derailleurs)
1,0.009091,(Caps)
11,0.018182,(Shorts)
4,0.027273,(Helmets)
8,0.027273,(Pedals)
3,0.036364,(Gloves)


In [28]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
#rules
rules.sort_values(by=['lift'], ascending = False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction'], axis=1)

#Lift > 1 means that there's a significative association rule

,antecedents,consequents,support,confidence,lift
